In [1]:
from forcefield import CGForceField
from model_builder import CGModelBuilder

In [2]:
import openmm.app as app

builder = CGModelBuilder(
    smiles_sequence="[OH][EO]10[OH]",
    bond_length=0.35,  # nm
    min_dist=0.35,  # nm
    max_tries=50000,
)
topology, positions = builder.create_molecule()

with open("peo10.pdb", "w") as f:
    app.PDBFile.writeFile(topology, positions, f)


In [3]:
structures = [
    {
        "pdb": "peo10.pdb",
        "number": 10,
        "box": (0, 0, 0, 50, 50, 50),  # Aを10個  50Å立方
    },
]

topology, positions = builder.create_packed_model(
    structures,
    output_pdb="peo10_10.pdb",  # 生成されるPACKMOL出力
)
pdb = app.PDBFile("peo10_10.pdb")
topology = pdb.topology
positions = pdb.positions

In [ ]:
param_dict = {
    "BOND": {
        ("EO", "EO"): {"r0": 0.5, "k": 100},
        ("EO", "OH"): {"r0": 0.45, "k": 120},
    },
    "NONBONDED": {
        "EO": {"sigma": 0.3, "epsilon": 0.5, "charge": 0.0, "mass": 72.0},
        "OH": {"sigma": 0.25, "epsilon": 0.4, "charge": -0.2, "mass": 18.0},
    },
    "ANGLE": {("EO", "EO", "EO"): {"theta0": 0.5, "k": 100}},
    "TORSION": {("EO", "EO", "EO", "EO"): {"periodicity": 3, "phase": 0.0, "k": 5.0}},
}
ff = CGForceField.from_dict(param_dict)
system = ff.create_system(topology)

In [5]:
import sys

import openmm
from openmm import unit

temperature = 300 * unit.kelvin
friction = 1 / unit.picosecond
timestep = 0.001 * unit.picoseconds
integrator = openmm.LangevinIntegrator(temperature, friction, timestep)

simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.reporters.append(app.PDBReporter("example-traj-packmol.pdb", 1000))
simulation.reporters.append(
    app.StateDataReporter(
        sys.stdout,
        1000,
        step=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
        volume=True,
        density=True,
    )
)
simulation.minimizeEnergy()
simulation.step(10_000)


#"Step","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
1000,575.612884122571,312.7851729710978,208.99671875377976,8.0,1.5692094184792849
2000,676.139926153234,357.2231765444395,238.68929288246247,8.0,1.5692094184792849
3000,724.4984309524774,406.06540999233886,271.32468422871506,8.0,1.5692094184792849
4000,830.3191488501573,443.6458761230541,296.4351907017401,8.0,1.5692094184792849
5000,796.2700176695051,419.2987453406663,280.1669309365677,8.0,1.5692094184792849
6000,788.6707529787506,456.0873717755765,304.7483461594219,8.0,1.5692094184792849
7000,779.7334957657997,473.5000929378217,316.3831738365638,8.0,1.5692094184792849
8000,777.5843398285936,464.56611202092006,310.41366869888304,8.0,1.5692094184792849
9000,775.0469709836447,434.49945569489665,290.32373778448795,8.0,1.5692094184792849
10000,734.0240686031341,464.9455360881127,310.66719217733385,8.0,1.5692094184792849


In [6]:
import MDAnalysis as mda
import nglview as nv

bonds = list(topology.bonds())
bonds_nglview = []
for bond in bonds:
    bonds_nglview.append((bond[0].index, bond[1].index))
u = mda.Universe("example-traj-packmol.pdb")
u.add_TopologyAttr("bonds", bonds_nglview)
view = nv.show_mdanalysis(u)
view.add_ball_and_stick()
view.add_representation("ball+stick", colorScheme="resname")
view

NGLWidget(max_frame=9)